# Lab 2: Advanced Camera Transformations
## Inverse View Matrix, Animation, Multiple Cameras & Chain Transformations

---

## วัตถุประสงค์การเรียนรู้ (Learning Objectives)

เมื่อจบ Lab นี้ นักศึกษาจะสามารถ:
1. สร้างและใช้งาน Inverse View Matrix เพื่อแปลงกลับจาก Camera Space ไป World Space
2. สร้าง Animation ของกล้องที่เคลื่อนที่ในวงโคจร
3. จัดการระบบหลายกล้อง และเปรียบเทียบมุมมองจากกล้องต่างๆ
4. เข้าใจและ Implement Chain Transformations สำหรับ Robot Arm

---

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.animation as animation
from IPython.display import HTML, display
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('seaborn-v0_8-whitegrid')
np.set_printoptions(precision=4, suppress=True)

print("Libraries loaded successfully!")

In [ ]:
# ===== Utility Functions from Lab 1 =====

def to_homogeneous(point):
    """แปลงจุด 3D เป็น Homogeneous Coordinates"""
    return np.append(point, 1)

def from_homogeneous(point_h):
    """แปลงจาก Homogeneous Coordinates กลับเป็น 3D"""
    return point_h[:3] / point_h[3]

def rotation_x(theta):
    """Rotation matrix around X-axis"""
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[1, 0, 0], [0, c, -s], [0, s, c]])

def rotation_y(theta):
    """Rotation matrix around Y-axis"""
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[c, 0, s], [0, 1, 0], [-s, 0, c]])

def rotation_z(theta):
    """Rotation matrix around Z-axis"""
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[c, -s, 0], [s, c, 0], [0, 0, 1]])

def euler_to_rotation_matrix(roll, pitch, yaw):
    """สร้าง Rotation Matrix จาก Euler Angles (ZYX convention)"""
    return rotation_z(yaw) @ rotation_y(pitch) @ rotation_x(roll)

def create_view_matrix(R_c, C):
    """
    สร้าง View Matrix: V = [R_c^T | -R_c^T @ C]
                          [0     | 1          ]
    """
    V = np.eye(4)
    R_c_T = R_c.T
    V[:3, :3] = R_c_T
    V[:3, 3] = -R_c_T @ C
    return V

def transform_point(V, X_world):
    """แปลงจุดโดยใช้ View Matrix"""
    X_homo = to_homogeneous(X_world)
    X_cam_homo = V @ X_homo
    return from_homogeneous(X_cam_homo)

def draw_coordinate_frame(ax, R, origin, scale=1.0, label='', alpha=1.0):
    """วาด Coordinate Frame"""
    colors = ['r', 'g', 'b']
    for i in range(3):
        direction = R[:, i] * scale
        ax.quiver(origin[0], origin[1], origin[2],
                  direction[0], direction[1], direction[2],
                  color=colors[i], arrow_length_ratio=0.1,
                  linewidth=2, alpha=alpha)
    if label:
        ax.text(origin[0], origin[1], origin[2] + scale * 0.3, label,
                fontsize=10, fontweight='bold')

print("Utility functions loaded!")

---

# Part 1: Inverse View Matrix

## 1.1 ทฤษฎี

### View Matrix ทำอะไร?
- **View Matrix (V)**: แปลงจาก World Space → Camera Space
- **Inverse View Matrix (V⁻¹)**: แปลงจาก Camera Space → World Space

### การคำนวณ Inverse

จาก View Matrix:
$$V = \begin{bmatrix} R_c^T & -R_c^T C \\ 0 & 1 \end{bmatrix}$$

Inverse View Matrix:
$$V^{-1} = \begin{bmatrix} R_c & C \\ 0 & 1 \end{bmatrix}$$

### ทำไม Inverse ง่ายขนาดนี้?

1. สำหรับ Rotation Matrix: $R^{-1} = R^T$ ดังนั้น $(R_c^T)^{-1} = R_c$
2. Translation กลับด้าน: $-R_c^T C$ กลับเป็น $C$

In [ ]:
def create_inverse_view_matrix(R_c, C):
    """
    สร้าง Inverse View Matrix (Camera-to-World)
    
    V^{-1} = [R_c  C]
             [0    1]
    
    Parameters:
    -----------
    R_c : numpy.ndarray (3, 3)
        Camera Rotation Matrix
    C : numpy.ndarray (3,)
        Camera Position in World Space
    
    Returns:
    --------
    V_inv : numpy.ndarray (4, 4)
        Inverse View Matrix
    """
    V_inv = np.eye(4)
    V_inv[:3, :3] = R_c
    V_inv[:3, 3] = C
    return V_inv

def inverse_view_matrix_from_V(V):
    """
    คำนวณ Inverse View Matrix จาก View Matrix โดยตรง
    (ใช้ได้เมื่อไม่มี R_c และ C)
    """
    return np.linalg.inv(V)

def camera_to_world(X_cam, V_inv):
    """
    แปลงจุดจาก Camera Space กลับไป World Space
    
    Parameters:
    -----------
    X_cam : numpy.ndarray (3,)
        Point in Camera Coordinates
    V_inv : numpy.ndarray (4, 4)
        Inverse View Matrix
    
    Returns:
    --------
    X_world : numpy.ndarray (3,)
        Point in World Coordinates
    """
    X_cam_homo = to_homogeneous(X_cam)
    X_world_homo = V_inv @ X_cam_homo
    return from_homogeneous(X_world_homo)

In [ ]:
# ===== ทดสอบ Inverse View Matrix =====

# กำหนด Camera
C = np.array([5, 3, 4])
R_c = euler_to_rotation_matrix(0, np.radians(30), np.radians(45))

# สร้าง View Matrix และ Inverse
V = create_view_matrix(R_c, C)
V_inv = create_inverse_view_matrix(R_c, C)

# ทดสอบ: V @ V_inv = I
identity_test = V @ V_inv

print("View Matrix V:")
print(V)
print("\nInverse View Matrix V^{-1}:")
print(V_inv)
print("\nV @ V^{-1} = I:")
print(identity_test)
print(f"\nIs Identity? {np.allclose(identity_test, np.eye(4))}")

In [ ]:
# ===== ทดสอบ Round-Trip Transformation =====

# จุดใน World Space
X_world_original = np.array([2, 1, 3])

# แปลงไป Camera Space
X_cam = transform_point(V, X_world_original)

# แปลงกลับมา World Space
X_world_recovered = camera_to_world(X_cam, V_inv)

print("Round-Trip Test:")
print(f"Original World Point:  {X_world_original}")
print(f"After World→Camera:    {X_cam}")
print(f"After Camera→World:    {X_world_recovered}")
print(f"\nRecovered correctly? {np.allclose(X_world_original, X_world_recovered)}")

In [ ]:
def visualize_inverse_transform():
    """
    แสดงการแปลงไป-กลับระหว่าง World และ Camera Space
    """
    fig = plt.figure(figsize=(16, 5))
    
    # Camera setup
    C = np.array([4, 2, 3])
    R_c = euler_to_rotation_matrix(0, np.radians(20), np.radians(30))
    V = create_view_matrix(R_c, C)
    V_inv = create_inverse_view_matrix(R_c, C)
    
    # Test points
    points_world = np.array([
        [1, 0, 0],
        [0, 2, 1],
        [2, 1, 2],
    ])
    
    # Transform to camera space
    points_cam = np.array([transform_point(V, p) for p in points_world])
    
    # Transform back to world space
    points_recovered = np.array([camera_to_world(p, V_inv) for p in points_cam])
    
    # --- Plot 1: World Space (Original) ---
    ax1 = fig.add_subplot(131, projection='3d')
    ax1.set_title('1. World Space (Original)', fontsize=12, fontweight='bold')
    draw_coordinate_frame(ax1, np.eye(3), np.zeros(3), scale=2, label='World')
    draw_coordinate_frame(ax1, R_c, C, scale=1, label='Camera')
    ax1.scatter(*points_world.T, s=100, c='purple', marker='o')
    for i, p in enumerate(points_world):
        ax1.text(p[0]+0.2, p[1]+0.2, p[2], f'P{i+1}', fontsize=9)
    ax1.set_xlim([-2, 6]); ax1.set_ylim([-2, 6]); ax1.set_zlim([-2, 6])
    ax1.set_xlabel('X'); ax1.set_ylabel('Y'); ax1.set_zlabel('Z')
    
    # --- Plot 2: Camera Space ---
    ax2 = fig.add_subplot(132, projection='3d')
    ax2.set_title('2. Camera Space (V × P)', fontsize=12, fontweight='bold')
    draw_coordinate_frame(ax2, np.eye(3), np.zeros(3), scale=2, label='Camera')
    ax2.scatter(*points_cam.T, s=100, c='orange', marker='o')
    for i, p in enumerate(points_cam):
        ax2.text(p[0]+0.2, p[1]+0.2, p[2], f'P{i+1}\'', fontsize=9)
    ax2.set_xlim([-4, 4]); ax2.set_ylim([-4, 4]); ax2.set_zlim([-2, 6])
    ax2.set_xlabel('$X_{cam}$'); ax2.set_ylabel('$Y_{cam}$'); ax2.set_zlabel('$Z_{cam}$')
    
    # --- Plot 3: World Space (Recovered) ---
    ax3 = fig.add_subplot(133, projection='3d')
    ax3.set_title('3. World Space (V⁻¹ × P\')', fontsize=12, fontweight='bold')
    draw_coordinate_frame(ax3, np.eye(3), np.zeros(3), scale=2, label='World')
    draw_coordinate_frame(ax3, R_c, C, scale=1, label='Camera')
    ax3.scatter(*points_recovered.T, s=100, c='green', marker='o')
    for i, p in enumerate(points_recovered):
        ax3.text(p[0]+0.2, p[1]+0.2, p[2], f'P{i+1}', fontsize=9)
    ax3.set_xlim([-2, 6]); ax3.set_ylim([-2, 6]); ax3.set_zlim([-2, 6])
    ax3.set_xlabel('X'); ax3.set_ylabel('Y'); ax3.set_zlabel('Z')
    
    plt.tight_layout()
    plt.savefig('inverse_transform.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    # Verify
    print("\nVerification:")
    print("-" * 50)
    for i in range(len(points_world)):
        match = np.allclose(points_world[i], points_recovered[i])
        print(f"P{i+1}: Original={points_world[i]} → Recovered={points_recovered[i]} ✓" if match else "✗")

visualize_inverse_transform()

### Exercise 1.1: Inverse Transform Practice

จุด P อยู่ที่ตำแหน่ง $(2, -1, 5)$ ใน **Camera Space**

กล้องตั้งอยู่ที่ $(3, 3, 3)$ ใน World Space และหมุน 45° รอบแกน Y

จงหาตำแหน่งของจุด P ใน World Space

In [ ]:
# Exercise 1.1: เขียนโค้ดที่นี่

# Given
P_cam = np.array([2, -1, 5])  # Point in camera space
C_ex = np.array([3, 3, 3])    # Camera position
theta_y = np.radians(45)      # 45 degrees

# TODO: สร้าง Rotation Matrix
R_c_ex = None

# TODO: สร้าง Inverse View Matrix
V_inv_ex = None

# TODO: แปลง P_cam ไป World Space
P_world_ex = None

# print(f"Point in World Space: {P_world_ex}")

In [ ]:
# Solution 1.1
R_c_ex_sol = rotation_y(np.radians(45))
V_inv_ex_sol = create_inverse_view_matrix(R_c_ex_sol, C_ex)
P_world_ex_sol = camera_to_world(P_cam, V_inv_ex_sol)

print("Solution:")
print(f"Camera Rotation (45° around Y):")
print(R_c_ex_sol)
print(f"\nInverse View Matrix:")
print(V_inv_ex_sol)
print(f"\nP_cam = {P_cam}")
print(f"P_world = {P_world_ex_sol}")

---

# Part 2: Camera Animation (Orbit)

## 2.1 ทฤษฎี: Orbital Camera

กล้องที่โคจรรอบจุด origin ในวงกลม:

$$C(t) = \begin{bmatrix} r \cos(\omega t) \\ h \\ r \sin(\omega t) \end{bmatrix}$$

โดยที่:
- $r$ = รัศมีวงโคจร
- $h$ = ความสูงคงที่
- $\omega$ = ความเร็วเชิงมุม
- $t$ = เวลา

กล้องต้อง **มองไปที่ Origin** ตลอดเวลา (Look-At)

In [ ]:
def look_at(eye, target, up=np.array([0, 1, 0])):
    """
    สร้าง View Matrix จาก Look-At parameters
    
    Returns: V, R_c (View Matrix and Camera Rotation)
    """
    forward = target - eye
    forward = forward / np.linalg.norm(forward)
    
    right = np.cross(forward, up)
    right = right / np.linalg.norm(right)
    
    true_up = np.cross(right, forward)
    
    R_c = np.column_stack([right, true_up, -forward])
    V = create_view_matrix(R_c, eye)
    
    return V, R_c

def orbit_camera_position(t, radius=5, height=3, omega=1):
    """
    คำนวณตำแหน่งกล้องที่โคจรรอบ origin
    
    Parameters:
    -----------
    t : float
        Time (angle in radians)
    radius : float
        Orbital radius
    height : float
        Camera height
    omega : float
        Angular velocity
    
    Returns:
    --------
    C : numpy.ndarray (3,)
        Camera position
    """
    x = radius * np.cos(omega * t)
    y = height
    z = radius * np.sin(omega * t)
    return np.array([x, y, z])

In [ ]:
def create_orbit_animation():
    """
    สร้าง Animation ของกล้องที่โคจรรอบ Origin
    """
    fig = plt.figure(figsize=(14, 6))
    
    # Scene objects
    scene_points = np.array([
        [0, 0, 0],
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1],
        [1, 1, 1],
        [-1, 0.5, 0.5]
    ])
    point_colors = ['red', 'orange', 'yellow', 'green', 'blue', 'purple']
    
    # Animation parameters
    n_frames = 60
    radius = 6
    height = 4
    
    def draw_frame(frame):
        fig.clear()
        
        t = 2 * np.pi * frame / n_frames
        
        # Camera position and orientation
        C = orbit_camera_position(t, radius, height)
        target = np.array([0, 0, 0])
        V, R_c = look_at(C, target)
        
        # Transform points to camera space
        points_cam = np.array([transform_point(V, p) for p in scene_points])
        
        # --- World View ---
        ax1 = fig.add_subplot(121, projection='3d')
        ax1.set_title(f'World Space (t = {np.degrees(t):.0f}°)', fontsize=12, fontweight='bold')
        
        # World axes
        draw_coordinate_frame(ax1, np.eye(3), np.zeros(3), scale=2, label='World')
        
        # Camera
        draw_coordinate_frame(ax1, R_c, C, scale=1.5, label='Cam')
        
        # Camera frustum
        ax1.plot([C[0], target[0]], [C[1], target[1]], [C[2], target[2]], 
                 'c--', alpha=0.5, linewidth=1)
        
        # Orbit path
        orbit_t = np.linspace(0, 2*np.pi, 100)
        orbit_x = radius * np.cos(orbit_t)
        orbit_z = radius * np.sin(orbit_t)
        orbit_y = np.ones_like(orbit_t) * height
        ax1.plot(orbit_x, orbit_y, orbit_z, 'c-', alpha=0.3, linewidth=1)
        
        # Scene points
        for i, (p, c) in enumerate(zip(scene_points, point_colors)):
            ax1.scatter(*p, s=80, c=c, marker='o')
        
        ax1.set_xlim([-8, 8]); ax1.set_ylim([-8, 8]); ax1.set_zlim([-2, 8])
        ax1.set_xlabel('X'); ax1.set_ylabel('Y'); ax1.set_zlabel('Z')
        
        # --- Camera View ---
        ax2 = fig.add_subplot(122, projection='3d')
        ax2.set_title('Camera Space', fontsize=12, fontweight='bold')
        
        # Camera axes (at origin)
        draw_coordinate_frame(ax2, np.eye(3), np.zeros(3), scale=2, label='Camera')
        
        # Transformed points
        for i, (p, c) in enumerate(zip(points_cam, point_colors)):
            ax2.scatter(*p, s=80, c=c, marker='o')
        
        ax2.set_xlim([-6, 6]); ax2.set_ylim([-6, 6]); ax2.set_zlim([-2, 10])
        ax2.set_xlabel('$X_{cam}$'); ax2.set_ylabel('$Y_{cam}$'); ax2.set_zlabel('$Z_{cam}$')
        
        plt.tight_layout()
    
    ani = animation.FuncAnimation(fig, draw_frame, frames=n_frames, interval=100)
    plt.close()
    
    return ani

# สร้างและแสดง Animation
print("Creating animation... (this may take a moment)")
ani = create_orbit_animation()
HTML(ani.to_jshtml())

In [ ]:
# บันทึก Animation เป็น GIF (optional)
# ani.save('orbit_camera.gif', writer='pillow', fps=10)
# print("Animation saved as orbit_camera.gif")

### Exercise 2.1: Spiral Orbit

สร้าง function สำหรับกล้องที่เคลื่อนที่เป็นเกลียว (Spiral) โดยความสูงเพิ่มขึ้นเรื่อยๆ

In [ ]:
def spiral_camera_position(t, radius=5, height_start=1, height_rate=0.5):
    """
    คำนวณตำแหน่งกล้องที่เคลื่อนที่เป็นเกลียว
    
    Parameters:
    -----------
    t : float
        Time/angle parameter
    radius : float
        Spiral radius
    height_start : float
        Starting height
    height_rate : float
        Height increase per radian
    
    Returns:
    --------
    C : numpy.ndarray (3,)
        Camera position
    """
    # TODO: Implement spiral motion
    # x = radius * cos(t)
    # y = height_start + height_rate * t
    # z = radius * sin(t)
    pass

# Visualize spiral path
# t_vals = np.linspace(0, 4*np.pi, 100)
# positions = np.array([spiral_camera_position(t) for t in t_vals])
# fig = plt.figure(); ax = fig.add_subplot(projection='3d')
# ax.plot(*positions.T)

In [ ]:
# Solution 2.1
def spiral_camera_position_solution(t, radius=5, height_start=1, height_rate=0.5):
    x = radius * np.cos(t)
    y = height_start + height_rate * t
    z = radius * np.sin(t)
    return np.array([x, y, z])

# Visualize
t_vals = np.linspace(0, 4*np.pi, 200)
positions = np.array([spiral_camera_position_solution(t) for t in t_vals])

fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(projection='3d')
ax.plot(*positions.T, 'b-', linewidth=2, label='Spiral Path')
ax.scatter(*positions[0], s=100, c='green', marker='o', label='Start')
ax.scatter(*positions[-1], s=100, c='red', marker='o', label='End')
draw_coordinate_frame(ax, np.eye(3), np.zeros(3), scale=2)
ax.set_xlabel('X'); ax.set_ylabel('Y'); ax.set_zlabel('Z')
ax.set_title('Spiral Camera Path', fontsize=12, fontweight='bold')
ax.legend()
plt.tight_layout()
plt.show()

---

# Part 3: Multiple Cameras

## 3.1 ทฤษฎี

ในระบบ Multi-Camera:
- แต่ละกล้องมี View Matrix ของตัวเอง
- จุดเดียวกันใน World Space จะมีพิกัดต่างกันในแต่ละ Camera Space
- ใช้ในงาน Stereo Vision, Motion Capture, 3D Reconstruction

In [ ]:
class Camera:
    """
    คลาสสำหรับเก็บข้อมูลกล้อง
    """
    def __init__(self, name, position, target=None, rotation=None, color='cyan'):
        """
        Parameters:
        -----------
        name : str
            Camera name
        position : numpy.ndarray (3,)
            Camera position in world space
        target : numpy.ndarray (3,), optional
            Look-at target (if specified, rotation is computed)
        rotation : numpy.ndarray (3, 3), optional
            Camera rotation matrix (used if target is None)
        color : str
            Color for visualization
        """
        self.name = name
        self.position = np.array(position)
        self.color = color
        
        if target is not None:
            self.target = np.array(target)
            self.V, self.R_c = look_at(self.position, self.target)
        else:
            self.R_c = rotation if rotation is not None else np.eye(3)
            self.V = create_view_matrix(self.R_c, self.position)
            self.target = None
        
        self.V_inv = create_inverse_view_matrix(self.R_c, self.position)
    
    def world_to_camera(self, X_world):
        """แปลงจุดจาก World Space ไป Camera Space"""
        return transform_point(self.V, X_world)
    
    def camera_to_world(self, X_cam):
        """แปลงจุดจาก Camera Space ไป World Space"""
        X_cam_homo = to_homogeneous(X_cam)
        X_world_homo = self.V_inv @ X_cam_homo
        return from_homogeneous(X_world_homo)
    
    def __repr__(self):
        return f"Camera('{self.name}', pos={self.position})"

In [ ]:
def visualize_multi_camera_system():
    """
    แสดงระบบหลายกล้องและมุมมองจากแต่ละกล้อง
    """
    # สร้างกล้อง 4 ตัว รอบๆ scene
    cameras = [
        Camera("Front", position=[0, 3, 8], target=[0, 0, 0], color='red'),
        Camera("Right", position=[8, 3, 0], target=[0, 0, 0], color='green'),
        Camera("Back", position=[0, 3, -8], target=[0, 0, 0], color='blue'),
        Camera("Left", position=[-8, 3, 0], target=[0, 0, 0], color='orange'),
    ]
    
    # จุดทดสอบใน World Space
    test_points = np.array([
        [0, 0, 0],    # Origin
        [2, 0, 0],    # +X
        [0, 2, 0],    # +Y
        [0, 0, 2],    # +Z
        [1, 1, 1],    # Diagonal
    ])
    point_labels = ['O', 'X', 'Y', 'Z', 'D']
    point_colors = ['black', 'red', 'green', 'blue', 'purple']
    
    fig = plt.figure(figsize=(16, 12))
    
    # --- Main World View ---
    ax_world = fig.add_subplot(2, 3, (1, 4), projection='3d')
    ax_world.set_title('World Space (Top-Down View)', fontsize=12, fontweight='bold')
    
    # World axes
    draw_coordinate_frame(ax_world, np.eye(3), np.zeros(3), scale=3, label='World')
    
    # Draw cameras
    for cam in cameras:
        draw_coordinate_frame(ax_world, cam.R_c, cam.position, scale=1.5, alpha=0.7)
        ax_world.text(cam.position[0], cam.position[1]+1, cam.position[2], 
                      cam.name, fontsize=9, color=cam.color, fontweight='bold')
        # Line to target
        if cam.target is not None:
            ax_world.plot([cam.position[0], cam.target[0]],
                         [cam.position[1], cam.target[1]],
                         [cam.position[2], cam.target[2]],
                         color=cam.color, linestyle='--', alpha=0.3)
    
    # Draw test points
    for p, lbl, c in zip(test_points, point_labels, point_colors):
        ax_world.scatter(*p, s=100, c=c, marker='o')
        ax_world.text(p[0]+0.2, p[1]+0.2, p[2]+0.2, lbl, fontsize=10)
    
    ax_world.set_xlim([-10, 10]); ax_world.set_ylim([-2, 10]); ax_world.set_zlim([-10, 10])
    ax_world.set_xlabel('X'); ax_world.set_ylabel('Y'); ax_world.set_zlabel('Z')
    ax_world.view_init(elev=30, azim=-60)
    
    # --- Camera Views ---
    subplot_positions = [(2, 3, 2), (2, 3, 3), (2, 3, 5), (2, 3, 6)]
    
    for cam, pos in zip(cameras, subplot_positions):
        ax = fig.add_subplot(*pos, projection='3d')
        ax.set_title(f'{cam.name} Camera View', fontsize=10, fontweight='bold', color=cam.color)
        
        # Camera frame at origin
        draw_coordinate_frame(ax, np.eye(3), np.zeros(3), scale=2)
        
        # Transform and draw points
        for p, lbl, c in zip(test_points, point_labels, point_colors):
            p_cam = cam.world_to_camera(p)
            ax.scatter(*p_cam, s=80, c=c, marker='o')
            ax.text(p_cam[0]+0.1, p_cam[1]+0.1, p_cam[2]+0.1, lbl, fontsize=8)
        
        ax.set_xlim([-5, 5]); ax.set_ylim([-5, 5]); ax.set_zlim([-2, 10])
        ax.set_xlabel('$X_c$'); ax.set_ylabel('$Y_c$'); ax.set_zlabel('$Z_c$')
    
    plt.tight_layout()
    plt.savefig('multi_camera_system.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    # Print point coordinates in each camera
    print("\n" + "="*70)
    print("Point Coordinates in Each Camera Space")
    print("="*70)
    print(f"{'Point':<8}", end='')
    for cam in cameras:
        print(f"{cam.name:<20}", end='')
    print()
    print("-"*70)
    
    for p, lbl in zip(test_points, point_labels):
        print(f"{lbl:<8}", end='')
        for cam in cameras:
            p_cam = cam.world_to_camera(p)
            print(f"({p_cam[0]:5.2f},{p_cam[1]:5.2f},{p_cam[2]:5.2f})  ", end='')
        print()

visualize_multi_camera_system()

### Exercise 3.1: Stereo Vision Setup

สร้างระบบ Stereo Vision ด้วยกล้อง 2 ตัว:
- **Left Camera**: อยู่ที่ $(-0.5, 0, 5)$
- **Right Camera**: อยู่ที่ $(0.5, 0, 5)$
- ทั้งคู่มองไปที่ Origin

คำนวณ **Disparity** (ความแตกต่างของพิกัด X ในแต่ละกล้อง) สำหรับจุดที่ต่างความลึก

In [ ]:
# Exercise 3.1: Stereo Vision

# TODO: สร้างกล้อง Left และ Right
cam_left = None
cam_right = None

# จุดที่ความลึกต่างกัน (Z ต่างกัน)
test_depths = [1, 2, 3, 4, 5]
disparities = []

# TODO: คำนวณ disparity สำหรับแต่ละความลึก
# for z in test_depths:
#     point = np.array([0, 0, -z])  # Point in front of cameras
#     left_x = cam_left.world_to_camera(point)[0]
#     right_x = cam_right.world_to_camera(point)[0]
#     disparity = left_x - right_x
#     disparities.append(disparity)

# print("Depth vs Disparity:")
# for z, d in zip(test_depths, disparities):
#     print(f"Z = {z}: disparity = {d:.4f}")

In [ ]:
# Solution 3.1
cam_left_sol = Camera("Left", position=[-0.5, 0, 5], target=[0, 0, 0], color='red')
cam_right_sol = Camera("Right", position=[0.5, 0, 5], target=[0, 0, 0], color='blue')

test_depths = [1, 2, 3, 4, 5]
disparities = []

print("Stereo Vision Analysis")
print("="*50)
print(f"{'Depth (Z)':<12} {'Left X':<12} {'Right X':<12} {'Disparity':<12}")
print("-"*50)

for z in test_depths:
    point = np.array([0, 0, -z])  # Point in front of cameras
    p_left = cam_left_sol.world_to_camera(point)
    p_right = cam_right_sol.world_to_camera(point)
    disparity = p_left[0] - p_right[0]
    disparities.append(disparity)
    print(f"{z:<12} {p_left[0]:<12.4f} {p_right[0]:<12.4f} {disparity:<12.4f}")

# Plot disparity vs depth
plt.figure(figsize=(8, 5))
plt.plot(test_depths, disparities, 'bo-', linewidth=2, markersize=10)
plt.xlabel('Depth (Z distance from cameras)', fontsize=12)
plt.ylabel('Disparity', fontsize=12)
plt.title('Disparity vs Depth (Stereo Vision)', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.show()

print("\nNote: Disparity ลดลงเมื่อวัตถุอยู่ไกลขึ้น (inverse relationship)")

---

# Part 4: Chain Transformations (Robot Arm)

## 4.1 ทฤษฎี: Forward Kinematics

### Robot Arm with Multiple Joints

สำหรับแขนหุ่นยนต์ที่มีหลายข้อต่อ:
- แต่ละข้อต่อมี Transformation Matrix ของตัวเอง
- ตำแหน่งสุดท้าย (End Effector) คำนวณจากการคูณ Matrix ต่อเนื่อง

$$T_{world \to end} = T_0 \cdot T_1 \cdot T_2 \cdot ... \cdot T_n$$

### DH Parameters (Denavit-Hartenberg)

แต่ละข้อต่อมี 4 parameters:
- $\theta$ = มุมหมุนรอบแกน Z
- $d$ = ระยะเลื่อนตามแกน Z  
- $a$ = ความยาวแขน (ตามแกน X)
- $\alpha$ = มุมบิดรอบแกน X

In [ ]:
def dh_transform(theta, d, a, alpha):
    """
    สร้าง Transformation Matrix จาก DH Parameters
    
    Parameters:
    -----------
    theta : float
        Joint angle (rotation around Z)
    d : float
        Link offset (translation along Z)
    a : float
        Link length (translation along X)
    alpha : float
        Link twist (rotation around X)
    
    Returns:
    --------
    T : numpy.ndarray (4, 4)
        Homogeneous Transformation Matrix
    """
    ct, st = np.cos(theta), np.sin(theta)
    ca, sa = np.cos(alpha), np.sin(alpha)
    
    T = np.array([
        [ct, -st*ca,  st*sa, a*ct],
        [st,  ct*ca, -ct*sa, a*st],
        [0,   sa,     ca,    d   ],
        [0,   0,      0,     1   ]
    ])
    return T

def simple_joint_transform(theta, length, axis='z'):
    """
    สร้าง Transformation Matrix แบบง่าย (หมุน + เลื่อน)
    
    Parameters:
    -----------
    theta : float
        Rotation angle (radians)
    length : float
        Link length
    axis : str
        Rotation axis ('x', 'y', or 'z')
    
    Returns:
    --------
    T : numpy.ndarray (4, 4)
        Transformation Matrix
    """
    T = np.eye(4)
    
    if axis == 'x':
        T[:3, :3] = rotation_x(theta)
        T[0, 3] = length
    elif axis == 'y':
        T[:3, :3] = rotation_y(theta)
        T[1, 3] = length
    else:  # z
        T[:3, :3] = rotation_z(theta)
        T[2, 3] = length
    
    return T

In [ ]:
class RobotArm:
    """
    คลาสสำหรับ Robot Arm ที่มีหลาย Joints
    """
    def __init__(self, link_lengths):
        """
        Parameters:
        -----------
        link_lengths : list of float
            Length of each link
        """
        self.link_lengths = link_lengths
        self.n_joints = len(link_lengths)
        self.joint_angles = [0.0] * self.n_joints
    
    def set_joint_angles(self, angles):
        """
        ตั้งค่ามุมของแต่ละ Joint
        
        Parameters:
        -----------
        angles : list of float
            Joint angles in radians
        """
        if len(angles) != self.n_joints:
            raise ValueError(f"Expected {self.n_joints} angles, got {len(angles)}")
        self.joint_angles = list(angles)
    
    def get_joint_transforms(self):
        """
        คำนวณ Transformation Matrix ของแต่ละ Joint
        
        Returns:
        --------
        transforms : list of numpy.ndarray
            List of 4x4 transformation matrices
        """
        transforms = []
        for i, (theta, length) in enumerate(zip(self.joint_angles, self.link_lengths)):
            # สลับแกนหมุน: Joint คู่หมุนรอบ Z, Joint คี่หมุนรอบ Y
            axis = 'z' if i % 2 == 0 else 'y'
            T = simple_joint_transform(theta, length, axis)
            transforms.append(T)
        return transforms
    
    def forward_kinematics(self):
        """
        คำนวณ Forward Kinematics (Chain Transformation)
        
        Returns:
        --------
        joint_positions : list of numpy.ndarray
            Position of each joint in world space
        end_effector : numpy.ndarray (3,)
            Position of end effector
        T_total : numpy.ndarray (4, 4)
            Total transformation matrix
        """
        transforms = self.get_joint_transforms()
        
        # Chain multiply
        T_total = np.eye(4)
        joint_positions = [np.array([0, 0, 0])]  # Base at origin
        
        for T in transforms:
            T_total = T_total @ T
            # Extract position from transformation
            pos = T_total[:3, 3]
            joint_positions.append(pos.copy())
        
        end_effector = T_total[:3, 3]
        return joint_positions, end_effector, T_total
    
    def get_all_chain_transforms(self):
        """
        คืนค่า Cumulative Transformation Matrix ของแต่ละ Joint
        """
        transforms = self.get_joint_transforms()
        chain = [np.eye(4)]
        T_cumulative = np.eye(4)
        
        for T in transforms:
            T_cumulative = T_cumulative @ T
            chain.append(T_cumulative.copy())
        
        return chain

In [ ]:
def visualize_robot_arm(arm, title="Robot Arm"):
    """
    แสดง Robot Arm ใน 3D
    """
    joint_positions, end_effector, T_total = arm.forward_kinematics()
    chain_transforms = arm.get_all_chain_transforms()
    
    fig = plt.figure(figsize=(12, 5))
    
    # --- 3D View ---
    ax1 = fig.add_subplot(121, projection='3d')
    ax1.set_title(title, fontsize=12, fontweight='bold')
    
    # World frame
    draw_coordinate_frame(ax1, np.eye(3), np.zeros(3), scale=1, label='Base')
    
    # Draw links
    positions = np.array(joint_positions)
    ax1.plot(positions[:, 0], positions[:, 1], positions[:, 2], 
             'b-', linewidth=4, label='Links')
    
    # Draw joints
    colors = plt.cm.viridis(np.linspace(0, 1, len(joint_positions)))
    for i, (pos, T) in enumerate(zip(joint_positions, chain_transforms)):
        ax1.scatter(*pos, s=100, c=[colors[i]], marker='o')
        ax1.text(pos[0]+0.1, pos[1]+0.1, pos[2]+0.1, f'J{i}', fontsize=9)
        # Draw local frame (smaller)
        R = T[:3, :3]
        draw_coordinate_frame(ax1, R, pos, scale=0.3, alpha=0.5)
    
    # End effector
    ax1.scatter(*end_effector, s=150, c='red', marker='*', label='End Effector')
    
    ax1.set_xlabel('X'); ax1.set_ylabel('Y'); ax1.set_zlabel('Z')
    ax1.set_xlim([-5, 5]); ax1.set_ylim([-5, 5]); ax1.set_zlim([-1, 8])
    ax1.legend()
    
    # --- Info Panel ---
    ax2 = fig.add_subplot(122)
    ax2.axis('off')
    
    info_text = "Joint Configuration:\n"
    info_text += "-" * 40 + "\n"
    for i, (angle, length) in enumerate(zip(arm.joint_angles, arm.link_lengths)):
        info_text += f"Joint {i}: θ = {np.degrees(angle):6.1f}°, L = {length:.1f}\n"
    
    info_text += "\n" + "-" * 40 + "\n"
    info_text += f"End Effector Position:\n"
    info_text += f"  X = {end_effector[0]:.3f}\n"
    info_text += f"  Y = {end_effector[1]:.3f}\n"
    info_text += f"  Z = {end_effector[2]:.3f}\n"
    
    info_text += "\n" + "-" * 40 + "\n"
    info_text += "Total Transform Matrix:\n"
    for row in T_total:
        info_text += f"  [{row[0]:7.3f} {row[1]:7.3f} {row[2]:7.3f} {row[3]:7.3f}]\n"
    
    ax2.text(0.1, 0.95, info_text, transform=ax2.transAxes, fontsize=10,
             verticalalignment='top', fontfamily='monospace',
             bbox=dict(boxstyle='round', facecolor='lightgray', alpha=0.8))
    
    plt.tight_layout()
    return fig

In [ ]:
# ===== Example: 3-Joint Robot Arm =====

# สร้าง Robot Arm 3 ข้อต่อ
arm = RobotArm(link_lengths=[2, 2, 1.5])

# ตั้งค่ามุม: 45°, -30°, 60°
arm.set_joint_angles([
    np.radians(45),   # Base rotation (Z)
    np.radians(-30),  # Shoulder (Y)
    np.radians(60)    # Elbow (Z)
])

fig = visualize_robot_arm(arm, "3-Joint Robot Arm")
plt.savefig('robot_arm.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
def create_robot_animation():
    """
    สร้าง Animation ของ Robot Arm ที่เคลื่อนไหว
    """
    fig = plt.figure(figsize=(10, 8))
    
    arm = RobotArm(link_lengths=[2, 2, 1.5])
    n_frames = 60
    
    def draw_frame(frame):
        fig.clear()
        ax = fig.add_subplot(111, projection='3d')
        
        # Animate joint angles
        t = 2 * np.pi * frame / n_frames
        arm.set_joint_angles([
            0.5 * np.sin(t),                    # Base oscillates
            np.radians(-30) + 0.3 * np.sin(2*t), # Shoulder oscillates faster
            np.radians(45) + 0.4 * np.cos(t)    # Elbow follows
        ])
        
        joint_positions, end_effector, _ = arm.forward_kinematics()
        
        # Draw base frame
        draw_coordinate_frame(ax, np.eye(3), np.zeros(3), scale=1)
        
        # Draw arm
        positions = np.array(joint_positions)
        ax.plot(positions[:, 0], positions[:, 1], positions[:, 2], 
                'b-', linewidth=4)
        
        # Draw joints
        for i, pos in enumerate(joint_positions):
            ax.scatter(*pos, s=80, c='green', marker='o')
        
        # End effector with trail
        ax.scatter(*end_effector, s=150, c='red', marker='*')
        
        ax.set_xlim([-5, 5]); ax.set_ylim([-5, 5]); ax.set_zlim([-1, 6])
        ax.set_xlabel('X'); ax.set_ylabel('Y'); ax.set_zlabel('Z')
        ax.set_title(f'Robot Arm Animation (Frame {frame+1}/{n_frames})', 
                    fontsize=12, fontweight='bold')
    
    ani = animation.FuncAnimation(fig, draw_frame, frames=n_frames, interval=100)
    plt.close()
    return ani

print("Creating robot animation...")
robot_ani = create_robot_animation()
HTML(robot_ani.to_jshtml())

### Exercise 4.1: 4-DOF Robot Arm

สร้าง Robot Arm ที่มี 4 ข้อต่อ และหาตำแหน่ง End Effector เมื่อ:
- Link lengths: [1.5, 2.0, 1.5, 1.0]
- Joint angles: [30°, -45°, 60°, -30°]

In [ ]:
# Exercise 4.1

# TODO: สร้าง 4-DOF Robot Arm
arm_4dof = None

# TODO: ตั้งค่ามุม
# arm_4dof.set_joint_angles([...])

# TODO: คำนวณ Forward Kinematics
# joint_pos, end_eff, T = arm_4dof.forward_kinematics()
# print(f"End Effector: {end_eff}")

# TODO: Visualize
# visualize_robot_arm(arm_4dof, "4-DOF Robot Arm")

In [ ]:
# Solution 4.1
arm_4dof_sol = RobotArm(link_lengths=[1.5, 2.0, 1.5, 1.0])

arm_4dof_sol.set_joint_angles([
    np.radians(30),
    np.radians(-45),
    np.radians(60),
    np.radians(-30)
])

joint_pos_sol, end_eff_sol, T_sol = arm_4dof_sol.forward_kinematics()

print("4-DOF Robot Arm Solution")
print("="*50)
print(f"End Effector Position: {end_eff_sol}")
print(f"\nTotal Transformation Matrix:")
print(T_sol)

fig = visualize_robot_arm(arm_4dof_sol, "4-DOF Robot Arm (Solution)")
plt.show()

### Exercise 4.2: Workspace Analysis

วิเคราะห์ **Workspace** (พื้นที่ที่ End Effector ไปถึงได้) ของ Robot Arm 2 ข้อต่อ

In [ ]:
# Exercise 4.2: Workspace Analysis

def analyze_workspace(arm, angle_range=(-np.pi, np.pi), n_samples=20):
    """
    วิเคราะห์ Workspace ของ Robot Arm
    
    Parameters:
    -----------
    arm : RobotArm
        Robot arm object
    angle_range : tuple
        Min and max angle for each joint
    n_samples : int
        Number of samples per joint
    
    Returns:
    --------
    workspace_points : numpy.ndarray
        All reachable end effector positions
    """
    angles = np.linspace(angle_range[0], angle_range[1], n_samples)
    workspace_points = []
    
    # TODO: Implement workspace sampling
    # for theta1 in angles:
    #     for theta2 in angles:
    #         arm.set_joint_angles([theta1, theta2])
    #         _, end_eff, _ = arm.forward_kinematics()
    #         workspace_points.append(end_eff)
    
    pass

# TODO: Create 2-DOF arm and analyze workspace

In [ ]:
# Solution 4.2
def analyze_workspace_solution(arm, angle_range=(-np.pi, np.pi), n_samples=30):
    angles = np.linspace(angle_range[0], angle_range[1], n_samples)
    workspace_points = []
    
    if arm.n_joints == 2:
        for theta1 in angles:
            for theta2 in angles:
                arm.set_joint_angles([theta1, theta2])
                _, end_eff, _ = arm.forward_kinematics()
                workspace_points.append(end_eff.copy())
    elif arm.n_joints == 3:
        for theta1 in angles[::2]:  # Coarser sampling for 3 joints
            for theta2 in angles[::2]:
                for theta3 in angles[::2]:
                    arm.set_joint_angles([theta1, theta2, theta3])
                    _, end_eff, _ = arm.forward_kinematics()
                    workspace_points.append(end_eff.copy())
    
    return np.array(workspace_points)

# Create and analyze 2-DOF arm
arm_2dof = RobotArm(link_lengths=[2, 1.5])
workspace = analyze_workspace_solution(arm_2dof)

# Visualize workspace
fig = plt.figure(figsize=(12, 5))

# 3D view
ax1 = fig.add_subplot(121, projection='3d')
ax1.scatter(workspace[:, 0], workspace[:, 1], workspace[:, 2], 
            c=workspace[:, 2], cmap='viridis', s=2, alpha=0.5)
ax1.set_xlabel('X'); ax1.set_ylabel('Y'); ax1.set_zlabel('Z')
ax1.set_title('2-DOF Robot Arm Workspace (3D)', fontsize=12, fontweight='bold')

# Top-down view (XY projection)
ax2 = fig.add_subplot(122)
ax2.scatter(workspace[:, 0], workspace[:, 1], c=workspace[:, 2], 
            cmap='viridis', s=2, alpha=0.5)
ax2.set_xlabel('X'); ax2.set_ylabel('Y')
ax2.set_title('Workspace (Top View)', fontsize=12, fontweight='bold')
ax2.set_aspect('equal')
ax2.grid(True, alpha=0.3)

# Draw workspace boundary
max_reach = sum(arm_2dof.link_lengths)
min_reach = abs(arm_2dof.link_lengths[0] - arm_2dof.link_lengths[1])
circle_outer = plt.Circle((0, 0), max_reach, fill=False, color='red', linestyle='--', label=f'Max reach: {max_reach}')
circle_inner = plt.Circle((0, 0), min_reach, fill=False, color='blue', linestyle='--', label=f'Min reach: {min_reach}')
ax2.add_patch(circle_outer)
ax2.add_patch(circle_inner)
ax2.legend()

plt.tight_layout()
plt.savefig('workspace_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nWorkspace Statistics:")
print(f"  Max reach: {max_reach:.2f}")
print(f"  Min reach: {min_reach:.2f}")
print(f"  Total sample points: {len(workspace)}")

---

# Part 5: Summary

## สรุปสิ่งที่เรียนรู้

### 1. Inverse View Matrix
$$V^{-1} = \begin{bmatrix} R_c & C \\ 0 & 1 \end{bmatrix}$$
- แปลงจาก Camera Space กลับไป World Space
- ใช้คุณสมบัติ $R^{-1} = R^T$ ของ Rotation Matrix

### 2. Camera Animation
- Orbital Camera: $C(t) = [r\cos(\omega t), h, r\sin(\omega t)]$
- ใช้ Look-At เพื่อให้กล้องมองไปที่เป้าหมายตลอด

### 3. Multiple Cameras
- แต่ละกล้องมี View Matrix ของตัวเอง
- จุดเดียวกันมีพิกัดต่างกันในแต่ละ Camera Space
- Stereo Vision: Disparity ∝ 1/Depth

### 4. Chain Transformations
$$T_{total} = T_0 \cdot T_1 \cdot T_2 \cdot ... \cdot T_n$$
- Forward Kinematics: คำนวณตำแหน่งจากมุมข้อต่อ
- DH Parameters: วิธีมาตรฐานในการกำหนด Robot Joints

---

## References

1. Hartley & Zisserman - Multiple View Geometry
2. Siciliano et al. - Robotics: Modelling, Planning and Control
3. Craig - Introduction to Robotics: Mechanics and Control
4. OpenGL Camera Tutorial: https://learnopengl.com/Getting-started/Camera